# Automatic Source Retrieval

Done:
- Ontario
- Manitoba
- British Columbia
- New Brunswick

TODO:
- Nova Scotia
- Northwest Territories
- Nunavut
- Yukon
- Alberta
- Saskatchewan
- Prince Edward Island
- Quebec
- Newfoundland and Labrador

In [ ]:
!python3 -m pip install bs4
!python3 -m pip install feedparser

In [7]:
import requests
import urllib.request
import datetime
from bs4 import BeautifulSoup
import pandas as pd
import feedparser
from datetime import date
from datetime import datetime

In [8]:
today = date.today()
today_str = str(today).replace('-', '%2F')

print(today_str)
country = 'Canada'
src_cat = 'Government Website'
columns = ['start_date', 'country', 'region', 'subregion', 'source_url', 'source_category', 'source_title', 'source_full_text']

2020%2F07%2F08


# Ontario

Since Ontario shows the most recent news on the first page, the range
will need to continue to be expanded to capture all posts. Generally ~4 pages capture ~2 weeks.

In [18]:
%%time

base_url = 'https://news.ontario.ca/en/search?content_type=all&utf8=%E2%9C%93&date_range_end=' + today_str + '&date_range_start=2020%2F01%2F01&date_select=desc&page='
targets = [base_url + str(i) for i in range(1,4)]

region = 'Ontario'
subregion = ''

# Specific structure for news.contario.ca/archive
links = []
page = 1
while True:
    print('Searching page ', page)
    target = base_url + str(page)
    
    response = requests.get(target)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.findAll('article')
    
    if len(articles) == 0:
        print('No articles found.')
        break
        
    for article in articles:
        smallersoup = BeautifulSoup(str(article), "html.parser")
        link = smallersoup.findAll('a')[0]['href']
        title = smallersoup.findAll('a')[0].string
        pub_date = datetime.strptime(smallersoup.time.string.replace('.', ''), "%B %d, %Y %I:%M %p")

        response = requests.get(link)
        linksoup = BeautifulSoup(response.text, "html.parser")
        full_text = linksoup.article.text

        row = [pub_date, country, region, subregion, link, src_cat, title, full_text]
        links.append(row)
    
    page += 1
            

Searching page  1
Searching page  2
Searching page  3
Searching page  4
Searching page  5
Searching page  6
Searching page  7
Searching page  8
Searching page  9
Searching page  10
Searching page  11
Searching page  12
Searching page  13
Searching page  14
Searching page  15
Searching page  16
Searching page  17
Searching page  18
Searching page  19
Searching page  20
Searching page  21
Searching page  22
Searching page  23
Searching page  24
Searching page  25
Searching page  26
Searching page  27
Searching page  28
Searching page  29
Searching page  30
Searching page  31
Searching page  32
Searching page  33
Searching page  34
Searching page  35
Searching page  36
Searching page  37
Searching page  38
Searching page  39
Searching page  40
Searching page  41
Searching page  42
Searching page  43
Searching page  44
Searching page  45
Searching page  46
Searching page  47
Searching page  48
Searching page  49
Searching page  50
Searching page  51
Searching page  52
Searching page  53
Se

In [19]:
df = pd.DataFrame(links, columns=columns)

In [23]:
df.shape

(1086, 8)

In [24]:
df['start_date'] = pd.to_datetime(df['start_date']).dt.date

In [25]:
df.to_csv('sources/ontario.csv')

# Manitoba

Retrieve all news releases in 2020 for the Province of Manitoba.

In [ ]:
url_base = 'https://news.gov.mb.ca'
targets = [url_base + '/news/index.html?month=' + str(i) + '&year=2020&day=01&bgnG=GO&d=' for i in range(1,12)]

In [ ]:
%%time

region = 'Manitoba'
subregion = ''

links = []
for target in targets:
    print(target)
    if target.startswith(url_base): #manitoba
        response = requests.get(target)
        soup = BeautifulSoup(response.text, "html.parser")
        items = soup.findAll("div", {"class": "maincontent"})
        smallersoup = BeautifulSoup(str(items), "html.parser")
        for article in smallersoup.findAll('h2'):
            a = article.a
            relative_link = a['href']
            link = url_base + relative_link.split('..')[-1]
            title = a.string
        
            response = requests.get(link)
            linksoup = BeautifulSoup(response.text, "html.parser")
            
            date_text = linksoup.findAll("span", {"class": "article_date"})[0].string
            date = pd.to_datetime(date_text, format='%B %d, %Y')
            pub_date = date.strftime('%m/%d/%Y')
            
            full_text = linksoup.findAll("div", {"class": ""})[0].text

            
            row = [pub_date, country, region, subregion, link, src_cat, title, full_text]
            links.append(row)
            
            # Get this link and copy full text

In [ ]:
df = pd.DataFrame(links, columns=columns)
df.to_csv('sources/manitoba.csv')
df.shape

## British Columbia

In [ ]:
#    'https://news.gov.bc.ca/Search?fromDate=2020%2F01%2F01&toDate=2020%2F05%2F26&q='
region = 'British Columbia'
subregion = ''

query_url = 'https://news.gov.bc.ca/Search?FromDate=01/01/2020&Page='

In [ ]:
links = []

In [ ]:
%%time

page = 1
while True:
    print("Page ", page)
    target = query_url + str(page)
    response = requests.get(target)
    soup = BeautifulSoup(response.text, "html.parser")
    items = soup.findAll("div", {"class": "article"})
    
    if not items:
        # No articles found
        break
    
    for article in items:
        smallersoup = BeautifulSoup(str(article), "html.parser")

        #for article in smallersoup.findAll('div'):

        title = smallersoup.a.string

        date_text = smallersoup.findAll("div", {"class" : "item-date"})[0].string
        date = pd.to_datetime(date_text)
        pub_date = date.strftime('%m/%d/%Y')

        link = smallersoup.a['href']

        response = requests.get(link)
        linksoup = BeautifulSoup(response.text, "html.parser")
        get_article = linksoup.findAll("article")
        if get_article:
            full_text = get_article[0].text
        else:
            print("Couldn't retrieve full text for link: ", link)
            full_text = ""

        row = [pub_date, country, region, subregion, link, src_cat, title, full_text]
        links.append(row)

    page += 1

In [ ]:
df = pd.DataFrame(links, columns=columns)
df.to_csv('sources/britishcolumbia.csv')
df.shape

In [26]:
# Returns a 2D array containing articles from New Brunsiwck since a specified start date, by default Jan 1 2020
def new_brunswick_links(since=datetime(2020, 1, 1)):
    
    region = 'New Brunswick'
    sub_region = ''
    
    url_base = "https://www2.gnb.ca/"
    url = url_base + "content/gnb/en/news/recent_news.html?mainContent_par_newslist_start="
    start = 0
    links = []
    
    while True:
        print("Page {}".format(str(start // 25 + 1)))
        request = requests.get(url + str(start))
        soup = BeautifulSoup(request.content, "html.parser")

        article_div = soup.find('div', class_="none padded")
        article_soup = BeautifulSoup(str(article_div), 'html.parser')
        articles = article_soup.find_all('li')

        for article in articles:
            small_soup = BeautifulSoup(str(article), 'html.parser')
            ar_date_str = small_soup.find('span', class_="post_date")
            
            if ar_date_str: # ensure list entry corresponds to dated article
                # Date
                ar_date = datetime.strptime(ar_date_str.text, "%d %B %Y")
                
                if ar_date < since: # only collect data after Jan 1 2020
                    print("Stopping search at {}".format(ar_date))
                    return links 
                
                a = article.a
                # Title
                title = a.text
                # Body
                relative_link = a['href']
                link = url_base + relative_link
                article_page = requests.get(link)
                body_soup = BeautifulSoup(article_page.content, 'html.parser')
                body = body_soup.find('div', class_="articleBody").text
                
                row = [ar_date, country, region, sub_region, link, src_cat, title, body]
                links.append(row)
#                 print("{}: {}\n".format(ar_date, title))
                

        start += 25 # articles per page

In [27]:
%%time

df = pd.DataFrame(new_brunswick_links(), columns=columns)
df.shape

Page 1
Page 2
Page 3
Page 4
Page 5
Page 6
Page 7
Page 8
Page 9
Page 10
Page 11
Page 12
Stopping search at 2019-12-30 00:00:00
CPU times: user 22 s, sys: 653 ms, total: 22.7 s
Wall time: 2min 58s


(287, 8)

In [28]:
df.to_csv('sources/newbrunswick.csv')

In [35]:
# Returns a 2D array containing Nova Scotian articles since a specified start date, by default Jan 1 2020
def load_nova_scotia(since=datetime(2020, 1, 1)):
    
    region = 'Nova Scotia'
    sub_region = ''
    
    url_base = "https://novascotia.ca/news"
    page = 1
    
    links = []
    
    while True:
        url = url_base + "/search/?page=" + str(page)
        print("Searching page {}".format(page))
        
        request = requests.get(url)
        soup = BeautifulSoup(request.content, 'html.parser')
        
        titles = soup.find_all('dt', class_="RelTitle")
        summaries = soup.find_all('dd', class_="RelSummary")
        
        for title, summary in zip(titles, summaries):
            
            if title['lang'] == "fr": continue
                        
            ar_date = datetime.strptime(summary.time.text, "%B %d, %Y - %I:%M %p")
            
            if ar_date < since:
                print("Stopping search at date {}".format(ar_date))
                return links
            
            relative_link = title.a['href'].split('..', 1)[1]
            link = url_base + relative_link
            
            ar_request = requests.get(link)
            ar_soup = BeautifulSoup(ar_request.content, 'html.parser')
            body = ar_soup.find('div', {'id' : 'releaseBody'}).text
            
            row = [ar_date, country, region, sub_region, link, src_cat, title.text, body]
            links.append(row)

            
        page += 1

In [36]:
%%time

df = pd.DataFrame(load_nova_scotia(), columns=columns)
df.shape

Searching page 1
Searching page 2
Searching page 3
Searching page 4
Searching page 5
Searching page 6
Searching page 7
Searching page 8
Searching page 9
Searching page 10
Searching page 11
Searching page 12
Searching page 13
Searching page 14
Searching page 15
Searching page 16
Searching page 17
Searching page 18
Searching page 19
Searching page 20
Searching page 21
Searching page 22
Searching page 23
Searching page 24
Searching page 25
Searching page 26
Searching page 27
Searching page 28
Searching page 29
Searching page 30
Searching page 31
Stopping search at date 2019-12-31 11:27:00
CPU times: user 14.4 s, sys: 659 ms, total: 15.1 s
Wall time: 8min 51s


(424, 8)

In [37]:
df.to_csv('sources/novascotia.csv')